In [1]:
import qval as qv
import numpy as np
import random

In [2]:

bits = 8

def quantize_matrix(matrix, n):
    rows, cols = matrix.shape

    qmatrix = [[0 for _ in range(rows)] for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            qmatrix[i][j] = qv.quantize(matrix[i][j], n)

    return np.array(qmatrix)

def dequantize_matrix(qmatrix, n):
    rows, cols = qmatrix.shape

    matrix = [[0 for _ in range(rows)] for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            matrix[i][j] = qv.dequantize(qmatrix[i][j], n)

    return np.array(matrix)



In [3]:
def qMatMul(m1, m2, n):
    rows = m1.shape[0]
    cols = m2.shape[1]
    muls = m2.shape[0]

    m3 = np.array([[qv.qZERO(n) for _ in range(cols)] for _ in range(rows)], dtype=object)

    for i in range(rows):
        for j in range(cols):
            sum_ij = qv.qZERO(n)
            for k in range(muls):
                prod = qv.qMul(m1[i][k], m2[k][j])
                prev_sum = sum_ij
                sum_ij = qv.qAdd(sum_ij, qv.qfit(prod, len(sum_ij)))
                print(f"Adding: {qv.dequantize(prev_sum)} + {qv.dequantize(prod)}, i: {qv.dequantize(prev_sum) + qv.dequantize(prod)}, q:{qv.dequantize(sum_ij)}")
            m3[i][j] = qv.qfit(sum_ij, n)

    return m3



def MatMul(m1, m2):
    rows = m1.shape[0]
    cols = m2.shape[1]
    muls = m2.shape[0]

    m3 = np.array([[0 for _ in range(cols)] for _ in range(rows)], dtype=object)

    for i in range(rows):
        for j in range(cols):
            sum_ij = 0
            for k in range(muls):
                
                prod = m1[i][k] * m2[k][j]
                sum_ij += prod
            
            m3[i][j] = sum_ij

    return m3

def matrix_mean_error(matrix, qmatrix):
    error_sum = 0
    for i in range(3):
        for j in range(3):
            error_sum += matrix[i][j] - qmatrix[i][j]

    return error_sum / 9

In [4]:
mat1 = np.random.uniform(-1, 1, (3, 3))
mat2 = np.random.uniform(-1, 1, (3, 3))

qmat1 = quantize_matrix(mat1, bits)
qmat2 = quantize_matrix(mat2, bits)

qmultmat = qMatMul(qmat1, qmat2, bits)
qmultmat = dequantize_matrix(qmultmat, bits)
print(qmultmat)
multmat = MatMul(mat1, mat2)

print(multmat)
error = matrix_mean_error(multmat, qmultmat)
print(error)

Adding: 0.0 + -0.11023622047244094, i: -0.11023622047244094, q:-0.05511811023622047
Adding: -0.05511811023622047 + 0.14960629921259844, i: 0.09448818897637797, q:0.047244094488188976
Adding: 0.047244094488188976 + 0.047244094488188976, i: 0.09448818897637795, q:0.047244094488188976
Adding: 0.0 + 0.2677165354330709, i: 0.2677165354330709, q:0.13385826771653545
Adding: 0.13385826771653545 + -0.25196850393700787, i: -0.11811023622047243, q:-0.05511811023622047
Adding: -0.05511811023622047 + 0.0, i: -0.05511811023622047, q:-0.023622047244094488
Adding: 0.0 + 0.18110236220472442, i: 0.18110236220472442, q:0.08661417322834646
Adding: 0.08661417322834646 + 0.25984251968503935, i: 0.3464566929133858, q:0.1732283464566929
Adding: 0.1732283464566929 + 0.0, i: 0.1732283464566929, q:0.08661417322834646
Adding: 0.0 + -0.09448818897637795, i: -0.09448818897637795, q:-0.047244094488188976
Adding: -0.047244094488188976 + 0.25984251968503935, i: 0.21259842519685038, q:0.10236220472440945
Adding: 0.1023

In [5]:
multmat = MatMul(mat1, mat2)
multmat

array([[0.09494943937879943, 0.022233107393803565, 0.4546079439336986],
       [-0.17634214779852608, -0.24780074165402152, 0.6063199825355803],
       [-0.4126044788689236, 0.8157400083470063, 0.23912298632681453]],
      dtype=object)

In [6]:

print(qv.qMax_i(bits))
a = random.uniform(0, 1)
b = random.uniform(0, 1)
print(f"a: {a}, b: {b}, a+b: {a+b}")
qa = qv.quantize(a, bits)
qb = qv.quantize(b, bits)
print(f"qa: {qa}, qb: {qb}, qa+qb: {qv.qAdd(qa, qb)}")
print(f"qai: {qv.qToInt(qa)}, qbi: {qv.qToInt(qb)}, (qa+qb)i: {qv.qToInt(qv.qAdd(qa, qb))}")
print(f"dqa: {qv.dequantize(qa)}, dqb: {qv.dequantize(qb)}, d(qa+qb): {qv.dequantize(qv.qAdd(qa, qb), bits)}")
print(qv.dequantize(qv.qFromInt(qv.qToInt(qv.qAdd(qa, qb)), bits), bits))

127
a: 0.5903839748627109, b: 0.8603537209006025, a+b: 1.4507376957633134
qa: 01001011, qb: 01101101, qa+qb: 01011100
qai: 75, qbi: 109, (qa+qb)i: 92
dqa: 0.5905511811023622, dqb: 0.8582677165354331, d(qa+qb): 0.7244094488188977
0.7244094488188977


In [7]:
bits = 8
print(qv.qMax_i(bits))
print(f"a: {a}, b: {b}, a*b: {a*b}")
qa = qv.quantize(a, bits)
qb = qv.quantize(b, bits)
print(f"qa: {qa}, qb: {qb}, qa*qb: {qv.qMul(qa, qb)}")
print(f"qai: {qv.qToInt(qa)}, qbi: {qv.qToInt(qb)}, (qa*qb)i: {qv.qToInt(qv.qMul(qa, qb))}")
print(f"dqa: {qv.dequantize(qa)}, dqb: {qv.dequantize(qb)}, d(qa*qb): {qv.dequantize(qv.qMul(qa, qb), None)}")
print(qv.dequantize(qv.qFromInt(qv.qToInt(qv.qMul(qa, qb)), bits), bits))

127
a: 0.5903839748627109, b: 0.8603537209006025, a*b: 0.5079390495332211
qa: 01001011, qb: 01101101, qa*qb: 00111111
qai: 75, qbi: 109, (qa*qb)i: 63
dqa: 0.5905511811023622, dqb: 0.8582677165354331, d(qa*qb): 0.49606299212598426
0.49606299212598426
